In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
# Script to visualize demonstrations according to the timestamps
# First get the first timestamp of the xela sensor
# Then with given fps given get image frames - find closest tactile and image observations

In [3]:
from tactile_learning.datasets.preprocess import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [4]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, 'cam_0_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        # print('image_timestamps: {}'.format(np.asarray(image_metadata['timestamps'])[0]))
        # print('image_timestamps: {}'.format(np.asarray(image_metadata['timestamps'])[-1]))
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]
        # print('tactile_timestamps: {}'.format(tactile_timestamps[0]))
        # print('tactile_timestamps: {}'.format(tactile_timestamps[-1]))

    # Compare the timestamps and calculate how much we should divide the image_timestamps - it can differ
    # img_ts = decimal.Decimal(str(image_timestamps_array[0]))
    # img_decimal_num = len(str(image_timestamps_array[0]).split(".")[0])
    # print(img_decimal_num)
    # tactile_ts = decimal.Decimal(str(tactile_timestamps[0]))
    # tactile_decimal_num = tactile_ts.as_tuple()
    # tactile_decimal_num = len(str(tactile_timestamps[0]).split(".")[0])
    # print(tactile_decimal_num)
    # print('img_decimal_num: {}, tactile_decimal_num: {}, diff: {}'.format(
    #     img_decimal_num, tactile_decimal_num, img_decimal_num - tactile_decimal_num
    # ))
    # img_divider = 10**(abs(img_decimal_num - tactile_decimal_num))
    # img_divider = 100
    # print(img_divider)
    # image_timestamps = image_timestamps_array / img_divider
    # print(image_timestamps_array[0] / img_divider)

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)
        # print('tactile_timestamps[{}]: {}, image_timestamps[{}]: {}'.format(
        #     tactile_id, tactile_timestamps[tactile_id], image_id, image_timestamps[image_id]
        # ))

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices

    # print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))

In [5]:

def dump_states(root, tactile_indices, image_indices):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_0_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        # print('image_id:')
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

        # break
        # if i > 10:
        #     break


def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    # print('tactile_values.shape: {}'.format(tactile_values.shape))

    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    # cv2.imwrite(os.path.join(viz_dir, 'Camera Image.png', image))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [13]:
# root = '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_box_opening_1'
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_box_opening_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_hammer_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_book_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bottle_opening_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_reverse_peg_4',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_joystick_2'
# ]
roots = [
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_handle_grabbing_1',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bowl_handle_lifting_4',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bowl_handle_lifting_6',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/demonstration_11'
]

retrieval_fps = 5
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices)


tactile_indices: [0, 22, 43, 61, 80, 100, 120, 140, 161, 179, 199, 219, 239, 258, 278, 297, 317, 337, 357, 376, 396, 416, 436, 455, 474, 494, 514, 534, 553, 573, 593, 612, 632, 652, 672, 691, 709, 729, 749, 769, 788, 807, 827, 847, 866, 886, 906, 926, 945, 964, 983, 1004, 1024, 1042, 1062, 1083, 1103, 1123, 1141, 1161, 1181, 1201, 1220, 1240, 1260, 1280, 1299, 1318, 1338, 1358, 1377, 1397, 1417, 1437, 1457, 1477, 1496, 1516, 1535, 1555, 1578, 1594, 1614, 1634, 1654, 1672, 1692, 1712, 1732, 1751, 1771, 1791, 1811, 1829, 1848, 1868, 1889, 1908, 1926, 1947, 1966, 1986, 2004, 2025, 2045, 2065, 2085, 2103, 2123, 2144, 2164, 2182, 2202, 2222, 2243, 2261, 2282, 2300, 2320, 2340, 2359, 2378, 2398, 2418, 2438, 2457, 2477, 2496, 2516, 2536, 2555, 2575, 2594, 2614, 2636, 2654, 2673, 2693, 2713, 2733, 2752, 2771, 2792, 2812, 2832, 2850, 2870, 2890, 2910, 2929, 2948, 2969, 2989, 3008, 3028, 3047, 3067, 3086, 3106, 3126, 3146, 3166, 3186, 3206, 3224, 3244, 3264, 3284, 3304, 3323, 3342, 3362, 3382, 3

100%|██████████| 174/174 [03:14<00:00,  1.12s/it]


In [14]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, 'visualization.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/demonstration_11


[libx264 @ 0x556d3ab5d440] using SAR=1/1
[libx264 @ 0x556d3ab5d440] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x556d3ab5d440] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x556d3ab5d440] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/demon

In [ ]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
